## Add Scores File

written by Isobel Mawby (i.mawby1@lancaster.ac.uk)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Imports
</div>

In [ ]:
import sys
import os
sys.path.insert(0, os.getcwd()[0:len(os.getcwd()) - 8])
sys.path.insert(1, os.getcwd()[0:len(os.getcwd()) - 8] + '/PrimaryTier/')
sys.path.insert(2, os.getcwd()[0:len(os.getcwd()) - 8] + '/LaterTier/')

import uproot
import numpy as np
import math
import torch

import Utilities
import PrimaryTierFileHelper
import LaterTierFileHelper
import Models

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Please put the path to your validation file and set ouput file name
</div>

In [ ]:
fileName = sys.path[0] + '/files/hierarchy.root'
outputFile = sys.path[0] + '/files/networkScores.npz'

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Load models.
</div>

In [ ]:
# # Model paths
PRIMARY_TRACK_BRANCH_MODEL_PATH = sys.path[0] + '/models/primary_track_branch_model'
TRACK_TRACK_BRANCH_MODEL_PATH = sys.path[0] + '/models/track_track_branch_model'
TRACK_SHOWER_BRANCH_MODEL_PATH = sys.path[0] + '/models/track_shower_branch_model'

PRIMARY_TRACK_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/primary_track_classifier_model'
PRIMARY_SHOWER_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/primary_shower_classifier_model'
TRACK_TRACK_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/track_track_classifier_model'
TRACK_SHOWER_CLASSIFIER_MODEL_PATH = sys.path[0] + '/models/track_shower_classifier_model'

# Models
primary_track_branch_model = torch.jit.load(PRIMARY_TRACK_BRANCH_MODEL_PATH)
track_track_branch_model = torch.jit.load(TRACK_TRACK_BRANCH_MODEL_PATH)
track_shower_branch_model = torch.jit.load(TRACK_SHOWER_BRANCH_MODEL_PATH)

primary_track_classifier_model = torch.jit.load(PRIMARY_TRACK_CLASSIFIER_MODEL_PATH)
primary_shower_classifier_model = torch.jit.load(PRIMARY_SHOWER_CLASSIFIER_MODEL_PATH)
track_track_classifier_model = torch.jit.load(TRACK_TRACK_CLASSIFIER_MODEL_PATH)
track_shower_classifier_model = torch.jit.load(TRACK_SHOWER_CLASSIFIER_MODEL_PATH)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Pull out variables from trees
</div>

In [ ]:
# Event tree
with uproot.open(f"{fileName}:EventTree") as eventTree:
    eventBranches = eventTree.arrays()
    event_eventID = np.array(eventBranches["EventID"])
    event_nRecoParticles = np.array(eventBranches["NRecoParticles"])
    event_nPrimaryTrackLinks = np.array(eventBranches["NPrimaryTrackLinks"])
    event_nPrimaryShowerLinks = np.array(eventBranches["NPrimaryShowerLinks"])
    event_nLaterTierTrackTrackLinks = np.array(eventBranches["NLaterTierTrackTrackLinks"])
    event_nLaterTierTrackShowerLinks = np.array(eventBranches["NLaterTierTrackShowerLinks"])
    
# Primary tier track tree
primaryTrack_nEntries, primaryTrack_variables, primaryTrack_particleID, primaryTrack_trueVisibleGeneration, primaryTrack_trueVisibleParentID, primaryTrack_truePDG = \
    PrimaryTierFileHelper.ReadTreeForValidation(True, fileName, True)

# Primary tier shower tree  
primaryShower_nEntries, primaryShower_variables, primaryShower_particleID, primaryShower_trueVisibleGeneration, primaryShower_trueVisibleParentID, primaryShower_truePDG = \
    PrimaryTierFileHelper.ReadTreeForValidation(False, fileName, True)

# Later tier track-track
trackTrack_nEntries, trackTrack_childID, trackTrack_parentID, trackTrack_variables, trackTrack_minNuVertexSep, trackTrack_separation3D = \
    LaterTierFileHelper.ReadTreeForValidation(True, fileName, True)

# Later tier track-shower tree
trackShower_nEntries, trackShower_childID, trackShower_parentID, trackShower_variables, trackShower_minNuVertexSep, trackShower_separation3D = \
    LaterTierFileHelper.ReadTreeForValidation(False, fileName, True)                                                          

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Obtain model scores
</div>

In [ ]:
primaryTrack_scores = Models.GetClassificationScore(primary_track_branch_model, primary_track_classifier_model, Models.primary_track_n_links, \
                                                    Models.primary_tier_n_orientation_indep_vars, Models.primary_tier_n_orientation_dep_vars, primaryTrack_variables)

primaryShower_scores = primary_shower_classifier_model(torch.tensor(primaryShower_variables, dtype=torch.float)).reshape(-1).tolist()

trackShower_scores = Models.GetClassificationScore(track_shower_branch_model, track_shower_classifier_model, Models.track_shower_n_links, \
                                                   Models.later_tier_n_orientation_indep_vars, Models.later_tier_n_orientation_dep_vars, trackShower_variables)

trackTrack_scores = Models.GetClassificationScore(track_track_branch_model, track_track_classifier_model, Models.track_track_n_links, \
                                                  Models.later_tier_n_orientation_indep_vars, Models.later_tier_n_orientation_dep_vars, trackTrack_variables)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Package up into events
</div>

In [ ]:
# Event arrays
event_particleID = []
event_trueGen = []
event_trueParentID = []
event_truePDG = []
event_trackScore = []
event_primary_scores = []
event_laterTier_parentID = []
event_laterTier_childID = []
event_laterTier_minNuVertexSep = []
event_laterTier_separation3D = []
event_laterTier_scores = [] 
    
# Initialise counters    
iPrimaryTrack = 0
iPrimaryShower = 0
iTrackShower = 0    
iTrackTrack = 0

for iEvent in range(event_eventID.shape[0]) :    
    # primary track
    nPrimaryTrackElements = int(event_nPrimaryTrackLinks[iEvent] / Models.primary_track_n_links)    
    event_particleID.append(primaryTrack_particleID[iPrimaryTrack:iPrimaryTrack + nPrimaryTrackElements].tolist())
    event_trueGen.append(primaryTrack_trueVisibleGeneration[iPrimaryTrack:iPrimaryTrack + nPrimaryTrackElements].tolist())
    event_trueParentID.append(primaryTrack_trueVisibleParentID[iPrimaryTrack:iPrimaryTrack + nPrimaryTrackElements].tolist())
    event_truePDG.append(primaryTrack_truePDG[iPrimaryTrack:iPrimaryTrack + nPrimaryTrackElements].tolist())
    event_trackScore.append([1.0] * nPrimaryTrackElements)
    event_primary_scores.append(primaryTrack_scores[iPrimaryTrack:iPrimaryTrack + nPrimaryTrackElements]) 
    iPrimaryTrack += nPrimaryTrackElements    
    # primary shower
    nPrimaryShowerElements = int(event_nPrimaryShowerLinks[iEvent] / Models.primary_shower_n_links)
    event_particleID[-1].extend(primaryShower_particleID[iPrimaryShower:iPrimaryShower + nPrimaryShowerElements].tolist())
    event_trueGen[-1].extend(primaryShower_trueVisibleGeneration[iPrimaryShower:iPrimaryShower + nPrimaryShowerElements].tolist())
    event_trueParentID[-1].extend(primaryShower_trueVisibleParentID[iPrimaryShower:iPrimaryShower + nPrimaryShowerElements].tolist())
    event_truePDG[-1].extend(primaryShower_truePDG[iPrimaryShower:iPrimaryShower + nPrimaryShowerElements].tolist())
    event_trackScore[-1].extend([0.0] * nPrimaryShowerElements)
    event_primary_scores[-1].extend(primaryShower_scores[iPrimaryShower:iPrimaryShower + nPrimaryShowerElements]) 
    iPrimaryShower += nPrimaryShowerElements    
    # track-track
    nTrackTrackElements = int(event_nLaterTierTrackTrackLinks[iEvent] / Models.track_track_n_links)    
    event_laterTier_parentID.append(trackTrack_parentID[iTrackTrack:iTrackTrack + nTrackTrackElements].tolist())
    event_laterTier_childID.append(trackTrack_childID[iTrackTrack:iTrackTrack + nTrackTrackElements].tolist())
    event_laterTier_minNuVertexSep.append(trackTrack_minNuVertexSep[iTrackTrack:iTrackTrack + nTrackTrackElements].tolist())
    event_laterTier_separation3D.append(trackTrack_separation3D[iTrackTrack:iTrackTrack + nTrackTrackElements].tolist())
    event_laterTier_scores.append(trackTrack_scores[iTrackTrack:iTrackTrack + nTrackTrackElements])
    iTrackTrack += nTrackTrackElements    
    # track-shower
    nTrackShowerElements = int(event_nLaterTierTrackShowerLinks[iEvent] / Models.track_shower_n_links)    
    event_laterTier_parentID[-1].extend(trackShower_parentID[iTrackShower:iTrackShower + nTrackShowerElements].tolist())
    event_laterTier_childID[-1].extend(trackShower_childID[iTrackShower:iTrackShower + nTrackShowerElements].tolist())    
    event_laterTier_minNuVertexSep[-1].extend(trackShower_minNuVertexSep[iTrackShower:iTrackShower + nTrackShowerElements].tolist())
    event_laterTier_separation3D[-1].extend(trackShower_separation3D[iTrackShower:iTrackShower + nTrackShowerElements].tolist())        
    event_laterTier_scores[-1].extend(trackShower_scores[iTrackShower:iTrackShower + nTrackShowerElements])
    iTrackShower += nTrackShowerElements  

<div class="alert alert-block alert-info" style="font-size: 18px;">
    In order to save as numpy arrays, we have to pad the vectors. First create link masks. 
</div>

In [ ]:
# Primary links
primary_link_max_length = Utilities.get_max_length(event_particleID)
print('primary_link_max_length:', primary_link_max_length)
primary_link_mask_main = [Utilities.create_mask(entry, primary_link_max_length) for entry in event_particleID]
primary_link_mask_main = np.array(primary_link_mask_main)

# Later tier links
laterTier_link_max_length = Utilities.get_max_length(event_laterTier_parentID)
print('laterTier_link_max_length:', laterTier_link_max_length)
laterTier_link_mask_main = [Utilities.create_mask(entry, laterTier_link_max_length) for entry in event_laterTier_parentID]
laterTier_link_mask_main = np.array(laterTier_link_mask_main)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Now pad the arrays.
</div>

In [ ]:
primary_particleID_main = Utilities.process_array(event_particleID, primary_link_max_length)
primary_trueGen_main = Utilities.process_array(event_trueGen, primary_link_max_length)
primary_trueParentID_main = Utilities.process_array(event_trueParentID, primary_link_max_length)
primary_truePDG_main = Utilities.process_array(event_truePDG, primary_link_max_length)
primary_trackScore_main = Utilities.process_array(event_trackScore, primary_link_max_length)
primary_score_main = Utilities.process_array(event_primary_scores, primary_link_max_length)
laterTier_parentID_main = Utilities.process_array(event_laterTier_parentID, laterTier_link_max_length)
laterTier_childID_main = Utilities.process_array(event_laterTier_childID, laterTier_link_max_length)
laterTier_minNuVertexSep_main = Utilities.process_array(event_laterTier_minNuVertexSep, laterTier_link_max_length)
laterTier_separation3D_main = Utilities.process_array(event_laterTier_separation3D, laterTier_link_max_length)
laterTier_score_main = Utilities.process_array(event_laterTier_scores, laterTier_link_max_length)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Save!
</div>

In [ ]:
np.savez(outputFile, \
         primary_link_mask_main = primary_link_mask_main, \
         primary_particleID_main = primary_particleID_main, \
         primary_trueGen_main = primary_trueGen_main, \
         primary_trueParentID_main = primary_trueParentID_main, \
         primary_truePDG_main = primary_truePDG_main, \
         primary_trackScore_main = primary_trackScore_main, \
         primary_score_main = primary_score_main, \
         laterTier_link_mask_main = laterTier_link_mask_main, \
         laterTier_parentID_main = laterTier_parentID_main, \
         laterTier_childID_main = laterTier_childID_main, \
         laterTier_nuVertexSep_main = laterTier_minNuVertexSep_main, \
         laterTier_separation3D_main = laterTier_separation3D_main, \
         laterTier_score_main = laterTier_score_main)